# Projet
Ce notebook a pour but d'implémenter notre propre classe en alternative au module `heapq` inclus dans python. Puis de comparer la performance du tri par tas avec cette classe a d'autres algorithmes de tri

In [ ]:
# Imports
import heapq
import time
from typing import List
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Any

## Implementation de notre alternative a heapq

In [ ]:
class MyHeapQ:
    """This our, not so good, implementation of a heap queue. It is inspired by the built-in heapq module."""
    @staticmethod
    def check_heap(arr: List[Any]) -> bool:
        """Check if the list satisfies the heap property (optimized)"""
        size = len(arr)
        for i in range((size - 2) // 2 + 1):
            left = (i << 1) + 1
            right = left + 1
            ai = arr[i]
            if left < size and ai > arr[left]:
                return False
            if right < size and ai > arr[right]:
                return False
        return True

    @staticmethod
    def heapify(arr: List[int]) -> None:
        """Transform list into a heap, in-place (optimized)"""
        size = len(arr)
        for i in range((size - 2) // 2, -1, -1):
            index = i
            while True:
                left = (index << 1) + 1
                right = left + 1
                smallest = index
                if left < size and arr[left] < arr[smallest]:
                    smallest = left
                if right < size and arr[right] < arr[smallest]:
                    smallest = right
                if smallest == index:
                    break
                arr[index], arr[smallest] = arr[smallest], arr[index]
                index = smallest

    @staticmethod
    def heappop(arr: List[int]) -> int:
        """Pop and return the smallest item from the heap (optimized)"""
        if not arr:
            raise IndexError("pop from empty heap")
        min_value = arr[0]
        last_item = arr.pop()
        if arr:
            arr[0] = last_item
            size = len(arr)
            index = 0
            while True:
                left = (index << 1) + 1
                right = left + 1
                smallest = index
                if left < size and arr[left] < arr[smallest]:
                    smallest = left
                if right < size and arr[right] < arr[smallest]:
                    smallest = right
                if smallest == index:
                    break
                arr[index], arr[smallest] = arr[smallest], arr[index]
                index = smallest
        return min_value

    @staticmethod
    def heappush(arr: List[int], value: int) -> None:
        """Push a new item onto the heap (optimized)"""
        arr.append(value)
        index = len(arr) - 1
        item = arr[index]
        while index > 0:
            parent = (index - 1) >> 1
            parent_val = arr[parent]
            if parent_val <= item:
                break
            arr[index] = parent_val
            index = parent
        arr[index] = item


## Tests d'intégrité

In [ ]:
nb_tests = 20 # Number of tests to run
list_size = 1000 # Size of the list to test

for _ in range(nb_tests):
  my_list = np.random.randint(0, 100, size=list_size).tolist()
  MyHeapQ.heapify(my_list)
  assert MyHeapQ.check_heap(my_list), "The list is not a valid heap after heapify."
  value_to_push = 5
  MyHeapQ.heappush(my_list, value_to_push)
  assert MyHeapQ.check_heap(my_list), "The list is not a valid heap after heappush."
  MyHeapQ.heappop(my_list)
  assert MyHeapQ.check_heap(my_list), "The list is not a valid heap after heappop."
print(f"All {nb_tests} tests passed!")

## Comparaison avec les autres algorithmes de tri
Ici, nous comparons le tri par tas avec `heapq` de python, notre heapq, le tri par insertion et enfin le tri a bulle. Ce dernier n'est présent que sur le dernier graph pour garder le premier lisible. En effet, la vitesse de celui-ci est pitoyable.

In [ ]:
class Tri:
  @staticmethod
  def my_tri_tas(tas: List[int]) -> List[int]:
      """Sort a list using heap sort"""
      h: List[int] = []
      for value in tas:
          MyHeapQ.heappush(h, value)
      return [MyHeapQ.heappop(h) for _ in range(len(h))]

  @staticmethod
  def tri_tas_heapq(tas: List[int]) -> List[int]:
    """Sort a list using the built-in heapq module"""
    h: List[int] = []
    for value in tas:
      heapq.heappush(h, value)
    return [heapq.heappop(h) for _ in range(len(h))]

  @staticmethod
  def tri_insertion(tab):
    """Sort a list using insertion sort"""
    n = len(tab)
    for i in range(1, n):
        key = tab[i]
        j = i - 1
        while j >= 0 and key < tab[j]:
            tab[j + 1] = tab[j]
            j -= 1
        tab[j + 1] = key
    return tab

  @staticmethod
  def tri_bulle(tab):
    """Sort a list using bubble sort"""
    n = len(tab)
    for i in range(n):
      for j in range(0, n - i - 1):
        if tab[j] > tab[j + 1]:
          tab[j], tab[j + 1] = tab[j + 1], tab[j]
    return tab

listen = [10, 20, 50, 100, 200, 500, 1_000]
def make_graph(functions_to_test: List[str]=[]):
  """Function to plot the performance of different sorting algorithms from the Tri class."""
  chrono = {}
  for method in functions_to_test:
      chrono[method] = np.zeros(len(listen))

  for k, n in enumerate(listen):
      tab = np.random.randint(0, 100, size=n)
      for method in functions_to_test:
          t0 = time.time()
          Tri.__dict__[method](tab.tolist())
          t1 = time.time()
          chrono[method][k] = t1 - t0

  plt.figure(1)
  # Plotting results
  for method in functions_to_test:
      plt.plot(listen, chrono[method], "o", label=method)
  plt.xlabel("Taille du tableau")
  plt.ylabel("Temps (s)")
  plt.title("Comparaison des résultats")
  plt.legend()
  plt.show()

make_graph(["tri_tas_heapq", "my_tri_tas", "tri_insertion"])
make_graph(["tri_tas_heapq", "my_tri_tas", "tri_insertion", "tri_bulle"])

Nous pouvons donc conclure que le tri par tas est la méthode de tri la plus efficace des trois. 
En revanche, l'implémentation native de python du tas est plus optimisée que la notre.
Cela est du a de nombreux facteurs, notamment le fait que cette dernière est écrite en language `C` qui est bien plus adapté pour ce genre de problèmes.